In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from PIL import Image
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')


In [2]:
class CarDataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.df = df.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.img_dir, f"{row['ID']}_0.jpg")
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        target = np.log1p(row['price_TARGET'])
        return image, torch.tensor(target, dtype=torch.float32)


In [ ]:
train_df = pd.read_parquet('train_dataset.parquet')

train_data, val_data = train_test_split(train_df, test_size=0.2, random_state=42)

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = CarDataset(train_data, 'train_imgs', transform=transform)
val_dataset = CarDataset(val_data, 'train_imgs', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0)

model = models.efficientnet_b0(pretrained=True)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 1)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
best_val_loss = float('inf')

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for images, targets in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}'):
        images, targets = images.to(device), targets.to(device)
        
        optimizer.zero_grad()

        with torch.cuda.amp.autocast():
            outputs = model(images).squeeze()
            loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
    
    model.eval()
    val_loss = 0
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for images, targets in tqdm(val_loader):
            images, targets = images.to(device), targets.to(device)
            outputs = model(images).squeeze()
            loss = criterion(outputs, targets)
            val_loss += loss.item()
            
            all_preds.extend(outputs.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())
    
    train_loss /= len(train_loader)
    val_loss /= len(val_loader)
    
    y_val = np.expm1(np.array(all_targets))
    pred = np.expm1(np.array(all_preds))
    median_ape = np.median(np.abs((y_val - pred) / y_val))
    score = 1 / (1 + median_ape)
    
    print(f'Epoch {epoch+1}: Train Loss = {train_loss:.4f}, Val Loss = {val_loss:.4f}, Score = {score:.4f}')
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'resnet50_best.pth')
        print('Модель сохранена!')

print(f'\nОбучение завершено! Лучший Val Loss: {best_val_loss:.4f}')


In [4]:
model.load_state_dict(torch.load('resnet50_best.pth'))
model.eval()

sample_idx = 0
sample_image, sample_target = val_dataset[sample_idx]

with torch.no_grad():
    sample_image = sample_image.unsqueeze(0).to(device)
    prediction = model(sample_image).squeeze().cpu().item()
    
actual_price = np.expm1(sample_target.item())
predicted_price = np.expm1(prediction)

print(f'Реальная цена: {actual_price:.0f}')
print(f'Предсказанная цена: {predicted_price:.0f}')


Реальная цена: 246000
Предсказанная цена: 186547
